# カタカナの画像認識モデル
手書きカタカナ「アイウエオカキクケコサシスセソ」の15文字を高い精度で識別できるモデルを構築する

## MLFlow準備

In [1]:
import os
import configparser
import mlflow
import mlflow.sklearn
# ローカルDB方式の場合に必要。今回はローカルGUI方式のため不要
# import sqlite3

In [2]:
# MLFlowの設定ファイル読み込み
cfg = configparser.ConfigParser()
cfg.read('./config.ini', encoding='utf-8')
# 各種パスを指定
# DB_PATH = cfg['Path']['db_path']

['./config.ini']

In [3]:
# %% エクスペリメントの作成
# Artifactストレージの場所を指定
ARTIFACT_LOCATION = cfg['Path']['artifact_location']
# Experimentの生成
EXPERIMENT_NAME = 'cnn_tuning'
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:  # 当該Experiment存在しないとき、新たに作成
    experiment_id = mlflow.create_experiment(
                            name=EXPERIMENT_NAME,
                            artifact_location=ARTIFACT_LOCATION)
else: # 当該Experiment存在するとき、IDを取得
    experiment_id = experiment.experiment_id

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle

## データ準備

### 訓練データ&検証データの読み込み

In [5]:
train_data = np.load("../1_data/train_data.npy")
train_label = np.load("../1_data/train_label.npy")

In [6]:
# 正規化
train_data = (train_data - train_data.min()) / train_data.max()
train_data = train_data.astype('float32')
# print(train_data)

In [7]:
# 配列形式変更
train_data = train_data.reshape(-1, 28*28)
print("train_data.shape=", train_data.shape)

train_data.shape= (3000, 784)


In [8]:
train, valid, y_train, y_valid = train_test_split(train_data, train_label, 
                                                    test_size=0.3, random_state=1234,
                                                    shuffle=True
                                                   )

print(train.shape, valid.shape)

(2100, 784) (900, 784)


In [9]:
# 配列形式変更
train = train.reshape(-1, 1, 28, 28)
valid = valid.reshape(-1, 1, 28, 28)

In [10]:
print(f'学習用データの形状：{train.shape}')
print(f'学習用ラベルの形状：{y_train.shape}')
print(f'検証用データの形状：{valid.shape}')
print(f'検証用ラベルの形状：{y_valid.shape}')

学習用データの形状：(2100, 1, 28, 28)
学習用ラベルの形状：(2100, 15)
検証用データの形状：(900, 1, 28, 28)
検証用ラベルの形状：(900, 15)


## モデル構築・学習

In [11]:
batchnorm=False
if batchnorm:
    from common.deep_convnet import DeepConvNetBatchNorm as DeepConvNet
else:
    from common.deep_convnet import DeepConvNet

from common.trainer_onlineaugmented import Trainer

2022-10-08 23:01:27.702171: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-08 23:01:27.950337: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-08 23:01:27.950410: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-08 23:01:27.985775: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-08 23:01:29.083744: W tensorflow/stream_executor/platform/de

In [12]:
# sampled = 200 # 高速化のため
# train = train[:sampled]
# y_train = y_train[:sampled]

In [13]:
print("訓練データをセット")
print(f'trainのshape：{train.shape}')
print(f'y_trainのshape：{y_train.shape}')

訓練データをセット
trainのshape：(2100, 1, 28, 28)
y_trainのshape：(2100, 15)


In [14]:
# 乱数の固定
_random_seed = 42
np.random.seed(_random_seed)

In [15]:
# ネットワーク生成（モデル構築）
network = DeepConvNet()  
print("ネットワーク生成完了")


ネットワーク生成完了


In [16]:
aug_param = {
    'zoom_range': [0.7, 1.3],
    'rotation_range': 40,
    'vertical_flip': False,
    'horizontal_flip': False,
    'height_shift_range': 0.1,
    'width_shift_range': 0.1,
    'shear_range': 40,
    # 'channel_shift_range': 127
}
add_image_num = 420

In [17]:
# _epochs=4
# _mini_batch_size=420
# _optimizer='RMSprop'
# _optimizer_param={'lr':0.01, 'decay_rate':0.9}

In [18]:
_epochs=80
_mini_batch_size=420
_optimizer='RMSprop'
_optimizer_param={'lr':0.01, 'decay_rate':0.9}
# 学習
trainer = Trainer(network, train, y_train, valid, y_valid,
                  aug_param, add_image_num, 
                  epochs=_epochs, mini_batch_size=_mini_batch_size, 
                  optimizer=_optimizer, optimizer_param=_optimizer_param,
                  evaluate_sample_num_per_epoch=1000)

# trainer = Trainer(network, train, y_train, valid, y_valid,
#                   epochs=20, mini_batch_size=100,
#                   optimizer='Adam', optimizer_param={'lr':0.001},
#                   evaluate_sample_num_per_epoch=1000)

trainer.train()

Trainerクラスのインスタンスが無事生成されました
optimizer:<common.optimizer.RMSprop object at 0x7f4fdc2e5120>
総iter数：400 = エポック数：80, ミニバッチ数：5.0
進捗：0.0%, 訓練データの誤差：12.4
==epoch:1, train_acc:0.066, test_acc:0.0667==
進捗：0.2%, 訓練データの誤差：2.7
進捗：0.5%, 訓練データの誤差：2.71
進捗：0.8%, 訓練データの誤差：2.71
進捗：1.0%, 訓練データの誤差：2.71
進捗：1.2%, 訓練データの誤差：2.71
==epoch:2, train_acc:0.068, test_acc:0.0667==
進捗：1.5%, 訓練データの誤差：2.71
進捗：1.8%, 訓練データの誤差：2.71
進捗：2.0%, 訓練データの誤差：2.71
進捗：2.2%, 訓練データの誤差：2.71
進捗：2.5%, 訓練データの誤差：2.71
==epoch:3, train_acc:0.079, test_acc:0.0544==
進捗：2.8%, 訓練データの誤差：2.71
進捗：3.0%, 訓練データの誤差：2.71
進捗：3.2%, 訓練データの誤差：2.71
進捗：3.5%, 訓練データの誤差：2.71
進捗：3.8%, 訓練データの誤差：2.7
==epoch:4, train_acc:0.077, test_acc:0.0589==
進捗：4.0%, 訓練データの誤差：2.71
進捗：4.2%, 訓練データの誤差：2.71
進捗：4.5%, 訓練データの誤差：2.71
進捗：4.8%, 訓練データの誤差：2.71
進捗：5.0%, 訓練データの誤差：2.71
==epoch:5, train_acc:0.097, test_acc:0.0844==


KeyboardInterrupt: 

In [ ]:
train_acc = trainer.train_acc_list
train_loss = trainer.train_loss_list
valid_acc = trainer.test_acc_list

### MLFlowに記録

In [ ]:
mlflow.end_run()

# 管理IDを発行（コードの実行に対して）
with mlflow.start_run(experiment_id=experiment_id):

    # ハイパーパラメータ, 評価指標, 学習済みモデルをMLflowへ保存
    ## データ拡張のパラメータ
    mlflow.log_param("aug_param", aug_param)
    mlflow.log_param("add_image_num", add_image_num)
    ## 学習のパラメータ
    mlflow.log_param("epochs", _epochs)
    mlflow.log_param("mini_batch_size", _mini_batch_size)
    mlflow.log_param("optimizer", _optimizer)
    mlflow.log_param("optimizer_param", _optimizer_param)
    mlflow.log_param("batchnorm", batchnorm)
    mlflow.log_param("random_seed", _random_seed)
    ## 評価指標
    mlflow.log_metric("train_acc", train_acc[-1])
    mlflow.log_metric("train_loss", train_loss[-1])
    mlflow.log_metric("valid_acc", valid_acc[-1])
    # mlflow.log_param("", )

    
    # mlflow.log_param("filter_num", filter_num)
    # mlflow.log_param("filter_size", filter_size)
    # mlflow.log_param("weight_init_std", weight_init_std)

    # mlflow.log_model(tnet, "model")

# mlflowを終了
mlflow.end_run()

In [ ]:
print(len(train_acc))
print(len(train_loss))
print(len(valid_acc))

In [ ]:
# lossとaccuracyのグラフ化
df_log = pd.DataFrame({"train_loss":train_loss,
             # "valid_loss":valid_loss,
             "train_acc":train_acc,
             "valid_acc":valid_acc})

df_log.plot(style=['r-', 'b-', 'b--'])
plt.ylim([0,3])
plt.ylabel("Accuracy or loss")
plt.xlabel("epochs")
plt.show()

### 学習済みモデルの出力

In [42]:
# with open("katakana_model.pickle", "wb") as f:
#     pickle.dump(tnet, f)

In [43]:
network.save_params("katakana_model.pickle")
print("Saved Network Parameters!")

Saved Network Parameters!


## 誤答を確認する

In [45]:
print(network)
print(valid.shape)

(900, 1, 28, 28)


In [46]:
network.predict(valid)

array([[-10.01048513,  -6.57092927, -22.31079914, ...,   6.48576623,
        -10.46153824, -15.10864127],
       [-16.41993826, -21.49478571,   6.00406515, ..., -17.78268256,
        -12.06279736, -10.73445212],
       [-30.01175184, -25.78453843, -20.28555109, ..., -15.62475733,
        -20.96645594,  -9.80670294],
       ...,
       [ -4.67686146,   5.29008032, -12.27082244, ...,  -3.75721018,
        -10.99703748,  -9.40572712],
       [ -4.14520855,  -7.8437439 ,  -4.37235523, ...,  -8.16081732,
         -5.26712889,  -5.82723493],
       [-15.74929326, -13.77952663,  -8.49608855, ...,  -8.92146507,
        -13.15932414,  -5.90590662]])